In [88]:
# Import Libraries & Make Working Copy
import pandas as pd
import numpy as np 

master_file = r"C:\Users\HP\Downloads\KarmicSeed_Pricing_Assignment\data\processed\master_pricing_table_v1.csv"

master_df = pd.read_csv(master_file)

print("Master DataFrame loaded successfully!")
print("Shape:", master_df.shape)
# --- Step 0: Make a working copy ---
pricing_df = master_df.copy()

Master DataFrame loaded successfully!
Shape: (50, 59)


In [89]:
# Ensure Numeric Columns
pricing_df['total_unit_cost'] = pd.to_numeric(pricing_df['total_unit_cost'], errors='coerce').fillna(0)
pricing_df['target_gross_margin_%'] = pd.to_numeric(pricing_df['target_gross_margin_%'], errors='coerce').fillna(0)
pricing_df.head(3)


,sku,product_description,product_role,country_of_origin,fba_fee,storage_fee,handling_cost,cost_x,current_price,minimum_acceptable_margin_%,...,units_ordered,ordered_product_sales,conversion_rate,impressions,clicks,cost_y,sales_7d,sales_14d,acos_7d,acos_14d
0,MN-01,Rectangle Tray -14 x 10 Inch - Pk of 25,Core,India,15.11,0.44,0.75,16.0,38.9,0.2,...,587,24582.01,0.305570,131593,985,2762.06,14235.45,14258.43,0.194027,0.193714
1,MN-02,Rectangle Tray -12x10 - Pk of 25,Core,India,13.55,0.30,0.75,12.0,33.9,0.2,...,431,15678.89,0.233985,226158,1298,2286.92,11626.53,11664.02,0.196698,0.196066
2,MN-03,Oval Tray -15x10 Inch - Pk of 25,Core,India,13.94,0.34,0.75,15.0,34.9,0.2,...,196,7340.65,0.228172,74652,394,670.20,3647.85,3675.34,0.183725,0.182350


In [90]:
# Compute Margin-Based Price
target_margin = pricing_df['target_gross_margin_%'] / 100.0
pricing_df['price_margin'] = pricing_df['total_unit_cost'] / (1 - target_margin)
pricing_df.head(3)


,sku,product_description,product_role,country_of_origin,fba_fee,storage_fee,handling_cost,cost_x,current_price,minimum_acceptable_margin_%,...,ordered_product_sales,conversion_rate,impressions,clicks,cost_y,sales_7d,sales_14d,acos_7d,acos_14d,price_margin
0,MN-01,Rectangle Tray -14 x 10 Inch - Pk of 25,Core,India,15.11,0.44,0.75,16.0,38.9,0.2,...,24582.01,0.305570,131593,985,2762.06,14235.45,14258.43,0.194027,0.193714,32.413447
1,MN-02,Rectangle Tray -12x10 - Pk of 25,Core,India,13.55,0.30,0.75,12.0,33.9,0.2,...,15678.89,0.233985,226158,1298,2286.92,11626.53,11664.02,0.196698,0.196066,26.693427
2,MN-03,Oval Tray -15x10 Inch - Pk of 25,Core,India,13.94,0.34,0.75,15.0,34.9,0.2,...,7340.65,0.228172,74652,394,670.20,3647.85,3675.34,0.183725,0.182350,30.135474


In [91]:
# Apply Competitor Price Constraints
comp_min = pricing_df['lowest_competitor_price'].fillna(-np.inf)
comp_max = pricing_df['highest_competitor_price'].fillna(np.inf)
pricing_df['price_competitor'] = pricing_df['price_margin'].clip(lower=comp_min, upper=comp_max)
pricing_df['recommended_price'] = pricing_df['price_competitor']


In [92]:
# Inventory-Based Price Adjustment
low_inv_thresh = pricing_df['weeks_of_cover_t30'].quantile(0.20)
high_inv_thresh = pricing_df['weeks_of_cover_t30'].quantile(0.80)

pricing_df.loc[pricing_df['weeks_of_cover_t30'] >= high_inv_thresh, 'recommended_price'] *= 0.95
pricing_df.loc[pricing_df['weeks_of_cover_t30'] <= low_inv_thresh, 'recommended_price'] *= 1.05


In [93]:
# Ads Efficiency Adjustment
if 'acos_7d' in pricing_df.columns:
    target_acos = 0.30
    high_acos = pricing_df['acos_7d'] > target_acos
    pricing_df.loc[high_acos, 'recommended_price'] *= 0.98


In [94]:
#  Handle Zero or Negative Prices
pricing_df['recommended_price'] = pricing_df.apply(
    lambda row: max(row['recommended_price'], row['total_unit_cost'], 1.0),
    axis=1
)



In [95]:
# Round Prices & Review
pricing_df['recommended_price'] = pricing_df['recommended_price'].round(2)

summary_cols = [
    'sku', 'current_price', 'recommended_price', 
    'total_unit_cost', 'target_gross_margin_%',
    'weeks_of_cover_t30', 'conversion_rate', 'units_ordered', 'sell_through' , 'lowest_competitor_price' , 'highest_competitor_price'
]

pricing_df[summary_cols].head(10)



,sku,current_price,recommended_price,total_unit_cost,target_gross_margin_%,weeks_of_cover_t30,conversion_rate,units_ordered,sell_through,lowest_competitor_price,highest_competitor_price
0,MN-01,38.90,32.41,32.30,0.35,6,0.305570,587,1.33,31.95,44.90
1,MN-02,33.90,27.90,26.60,0.35,7,0.233985,431,1.56,27.90,39.95
2,MN-03,34.90,30.03,30.03,0.35,8,0.228172,196,1.16,29.99,42.50
3,MN-04,29.90,25.94,25.94,0.35,8,0.316225,191,1.65,24.50,35.90
4,MN-05,21.90,25.77,25.77,0.45,0,0.177882,341,1.47,18.95,0.00
5,MN-06,14.95,14.85,14.37,0.45,0,0.220641,124,1.23,12.99,21.50
6,MN-07,34.90,29.97,29.87,0.35,5,0.130028,236,1.25,28.90,41.95
7,MN-08,29.90,26.11,26.11,0.45,98,0.255952,43,0.37,25.95,39.90
8,MN-09,14.90,14.92,14.92,0.45,43,0.157282,81,0.36,11.99,22.90
9,MN-10,19.90,17.40,17.40,0.35,10,0.178019,230,1.17,16.95,26.90


In [96]:
# Inventory Thresholds for Reference
print(f"Low inventory threshold (20th percentile): {low_inv_thresh:.2f} weeks")
print(f"High inventory threshold (80th percentile): {high_inv_thresh:.2f} weeks")


Low inventory threshold (20th percentile): 4.00 weeks
High inventory threshold (80th percentile): 8.00 weeks


In [97]:
final_prices_df = pricing_df[summary_cols].copy()
final_prices_df.to_csv('../data/processed/recommended_prices_clean.csv', index=False)
